# Ages and Masses from GALAH Spectra with The Cannon

## Part 1: Creating a training set

In [16]:
# Preamble
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

from astropy.io import fits
import numpy as np
from astropy.table import Table, join
from scipy.io import readsav
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import pickle

# Read in and join GALAH DR3 and asteroseismic catalogs

In [17]:
# Read in GALAH DR3
galah_dr3 = Table.read('data/GALAH_DR3_all_joined_v2.fits')
galah_dr3 = galah_dr3[galah_dr3['dr3_source_id_1'] > 0]

In [18]:
# Read in the asteroseismic data from Zinn et al. (2022)
seismic = Table.read('data/Zinn_Table2_eDR3_xmatch.fits')

# Rename some keywords for ease
seismic['dr3_source_id_1'] = seismic['source_id']
seismic['nu_max'] = seismic['numax_mean']
seismic['delta_nu'] = seismic['dnu_mean_corr']

In [19]:
# Join GALAH DR3 and the asteroseismic data
data = join(galah_dr3, seismic, keys='dr3_source_id_1')

# Rename some keywords for ease
data['mass'] = data['m_act_bstep']
data['age'] = data['age_bstep']

# Calculate masses from scaling relations
data['mass_astero'] = (data['nu_max']/3076.)**3. * (data['delta_nu']/135.146)**(-4) * (data['teff']/5772)**(3./2.)

# Trainingset Label Selection

We have to do 2 steps, because not all spectra are available publicly for GALAH DR3  
So our approach is: select all useful catalog entries, then make sure they all have spectra, then save those as the trainingset

In [21]:
# Select a useful preliminary trainingset

selection = (
    (data['flag_sp'] == 0) & 
    (data['flag_fe_h'] == 0) & 
    (data['flag_alpha_fe'] == 0) & 
    (data['snr_c2_iraf'] > 75) & 
    (data['teff'] < 5150) & 
    np.isfinite(data['age']) & 
    np.isfinite(data['mass']) & 
    np.isfinite(data['numax_mean']) & 
    np.isfinite(data['dnu_mean_corr'])
)
print(len(data['sobject_id'][selection]))

989


In [22]:
# Create catalog of trainingset
preliminary_trainingset = Table()
for key in ['sobject_id', 'teff', 'logg','fe_h','alpha_fe','nu_max', 'delta_nu','mass','age','mass_astero','j_m','h_m','ks_m','snr_c2_iraf','Mg_fe','Mn_fe','Al_fe']:
    preliminary_trainingset[key] = data[key][selection]

In [25]:
# Save the training set ids, so that we can download them from datacentral.org.au
np.savetxt('data/trainingset_ids.txt',[", ".join(str(x) for x in preliminary_trainingset['sobject_id'])],fmt='%s')

# Now use this list for a bulk download (as described on galah-survey.org) from https://datacentral.org.au/services/download/

In [26]:
# Extract the spectra in observation/hermes/*sobejctid*ccd*.fits

# Now we check which spectra are missing
missing = []

for sobject_id in preliminary_trainingset['sobject_id']:
    
    try:
        for ccd in [1,2,3,4]:

            fits_file = fits.open('observation/hermes/'+str(sobject_id)+str(ccd)+'.fits')
            s = fits_file[4].data

            fits_file.close()
    except:
        missing.append(sobject_id)

# Vice versa: Which spectra are available?
available = []

for sobject_id in preliminary_trainingset['sobject_id']:
    if sobject_id not in missing:
        available.append(True)
    else:
        available.append(False)

In [28]:
# Now save the final trainingset for which we know all spectra are available

trainingset = preliminary_trainingset#[available]
final_trainingset = Table()
for key in ['sobject_id', 'teff', 'logg','fe_h','alpha_fe','nu_max', 'delta_nu','mass','age','mass_astero','j_m','h_m','ks_m','snr_c2_iraf','Mg_fe','Mn_fe','Al_fe']:
    final_trainingset[key] = trainingset[key]
    
final_trainingset.write('trainingset.fits',overwrite=True)
final_trainingset

sobject_id,teff,logg,fe_h,alpha_fe,nu_max,delta_nu,mass,age,mass_astero,j_m,h_m,ks_m,snr_c2_iraf,Mg_fe,Mn_fe,Al_fe
,K,log(cm.s**-2),,,,,,,,mag,mag,mag,,,,
int64,float32,float32,float32,float64,float32,float32,float64,float64,float32,float32,float32,float32,float64,float64,float64,float64
161006005401094,4830.7607,2.364735,-0.33675337,0.0800271859202082,32.167,4.337,1.0947040243794466,6.376832077049619,0.82558477,9.194,8.631,8.437,86.5558826768,0.08066501617431676,0.008977813720703054,0.10905775070190415
161006005401076,4672.3394,2.3231006,-0.115608215,0.049204728181834044,37.015,4.196,1.1224882870355306,6.593441477004973,1.3657057,8.827,8.198,8.045,119.784236576,0.032440090179443715,0.04616729736328118,0.19335111618041978
140824006301112,4691.825,2.4805117,-0.4296875,0.14221368473665205,41.132,4.94,0.944287707625185,10.9007192785821,0.98154163,9.137,8.467,8.319,120.893414938,0.16069593429565465,0.012030048370361257,0.27458461761474595
140824006301117,4584.031,2.1341462,-0.29746866,0.06269277936057815,35.526,3.929,1.1185212152459802,6.93231590248784,1.5263231,7.334,6.715,6.545,217.087372697,0.03424158096313512,-0.06255920410156257,0.12979396820068345
140824006301178,4868.853,2.492118,-0.15996265,0.04322181828260722,89.557,7.399,1.7387946464289636,2.03368636095098,2.1281917,8.63,8.068,7.885,103.787835267,-0.05573377609252894,0.07283918380737298,0.10179599761962876
151111003101006,4551.1636,1.9781973,-0.46227074,0.08734366140261351,15.636,2.479,1.079395123830893,7.143227732624391,0.8123041,7.72,7.017,6.845,104.120964512,0.08606328964233434,-0.08708627700805671,0.24604972839355455
151111003101382,4220.5767,1.9814734,-0.085291386,0.16184932646030029,18.663,2.508,1.0569430544009966,9.877934263582587,1.1774876,7.365,6.646,6.414,85.523630349,0.17622365951538121,--,0.3534611129760741
160923004201116,4466.065,2.2739487,-0.011614323,0.10969648273252203,31.858,3.906,1.0849137737086834,8.455245029001855,1.0836236,9.181,8.548,8.365,77.6402157905,0.2609666824340824,0.08653490066528313,0.30827650070190415


# Trainingset Wavelength, Flux, and InverseVariance Preparation

Now that we have saved all labels, we need to prepare the wavelengths, fluxes, and inverse variance (ivar) values for the training set

In [ ]:
# Wavelength array
# For GALAH, we have 4 CCDs

cannon_wave = dict()
cannon_wave['1']=np.arange(4715.94,4896.00,0.046) # ab lines 4716.3 - 4892.3
cannon_wave['2']=np.arange(5653.31,5868.25,0.055) # ab lines 5646.0 - 5867.8
cannon_wave['3']=np.arange(6480.52,6733.92,0.064) # ab lines 6481.6 - 6733.4
cannon_wave['4']=np.arange(7726.95,7875.55,0.074) # ab lines 7691.2 - 7838.5

wavelength_array = np.concatenate(([cannon_wave[key] for key in ['1','2','3','4']]))
np.savetxt('wavelength.txt',wavelength_array,fmt='%s')

In [ ]:
# The Cannon needs FLUX and IVAR as inputs
# Lets prepare empty arrays and then fill them by looping through the individual training set entries

normalized_flux = np.ones((np.shape(final_trainingset)[0],np.shape(wavelength_array)[0]))
normalized_ivar = np.ones((np.shape(final_trainingset)[0],np.shape(wavelength_array)[0]))

for index, sobject_id in enumerate(final_trainingset['sobject_id']):
    
    normalised_flux_for_index = []
    normalised_ivar_for_index = []
    
    for ccd in [1,2,3,4]:
        
        fits_file = fits.open('observation/hermes/'+str(sobject_id)+str(ccd)+'.fits')
        wave = fits_file[4].header['CRVAL1'] + fits_file[4].header['CDELT1'] * np.arange(fits_file[4].header['NAXIS1'])
        flux = fits_file[4].data
        flux_unc = fits_file[4].data * fits_file[1].data
        
        # Not all spectra are already on the Cannon wavelength grid, so we have to interpolate them onto the Cannon wavelength grid
        flux_interpolated = np.interp(cannon_wave[str(ccd)], wave, flux)
        flux_unc_interpolated = np.interp(cannon_wave[str(ccd)], wave, flux_unc)
        
        normalised_flux_for_index.append(flux_interpolated)
        normalised_ivar_for_index.append(1./(flux_unc_interpolated**2))
        
    normalised_flux_for_index = np.concatenate((normalised_flux_for_index))
    normalised_ivar_for_index = np.concatenate((normalised_ivar_for_index))
    
    normalized_flux[index] = normalised_flux_for_index
    normalized_ivar[index] = normalised_ivar_for_index
    
# Now save these 2 entries to a pickle file (more efficient storage)
flux_ivar_file_opener = open('trainingset_flux_ivar.pickle','wb')
pickle.dump((normalized_flux,normalized_ivar),flux_ivar_file_opener)
flux_ivar_file_opener.close()